In [2]:
import pandas as pd
import numpy as np

In [2]:
from __future__ import print_function
from os.path import dirname
import os,sys,inspect

current_dir = dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
manage_dir = dirname(dirname(dirname(current_dir)))

if not manage_dir in sys.path:
    sys.path.insert(0, manage_dir)

# from pyml4.common import context, db
# context.local_context.print_summary()

import gensim
from gensim.models import Word2Vec
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import logging
import nltk
import nltk.data
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import SnowballStemmer
from nltk.corpus import webtext
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from nltk.corpus import stopwords
import pymysql.cursors

In [6]:
tag_no_cap_punc_stem = pd.read_csv("tag_no_cap_punc_stem_dup.csv")

In [7]:
tag_no_cap_punc_stem.head()

,tag,movieId,score,tag_stem,tags_same_stem,duplicates
0,sci-fi,30393804,7842.0,sci-fi,NaN,Sci-Fi;Sci-fi;#scifi;+scifi;
1,atmospheric,22351413,5399.0,atmospher,Atmosphere;,ATMOSPHERIC;Atmospheric;
2,surreal,30372701,4581.0,surreal,surrealism;,Surreal;
3,twist ending,14940576,4102.0,twist end,twist endings;twist end;,Twist Ending;Twist ending;
4,visually appealing,20032372,3866.0,visually app,NaN,Visually Appealing;Visually appealing;


In [9]:
tag_list = tag_no_cap_punc_stem['tag'].tolist()

In [5]:
word_vec_model2 = Word2Vec.load('../models/Word2Vec_fullds_model2')

In [15]:
word2vec = []
word2vec_2edits = []

for index, row in tag_no_cap_punc_stem.iterrows():
    tag = row[0]
    temp1 = ''
    temp2 = ''
    if tag in word_vec_model2.wv.vocab:
        for (k,v) in word_vec_model2.wv.similar_by_word(tag):
            if k in tag_list:
                #temp.append(k)
                temp1 += k+";"
                if isEditDistanceTwo(tag,k):
                    temp2 += k+";"
                
    if len(temp1) > 0:
        word2vec.append(temp1)
    else:
        word2vec.append(None)
    if len(temp2) > 0:
        word2vec_2edits.append(temp2)
        print(temp2)
    else:
        word2vec_2edits.append(None)

/Users/evazhong/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


superheros;
cinematograpy;
psycological;
musics;
mistery;
philosphy;
sequels;sequal;
robot;
prision;
orginal;
gory;
assassins;assasin;
vampire;
surrealist;
humourous;
monsters;
hillarious;
vampires;
1990s;1970s;1950s;1960s;1980;
journalist;
terrorist;
adultry;
computer;
1990s;1960s;1950s;1980s;1970;
intellectuals;
islands;
genious;
1950s;1970s;1980s;1990s;1960;
suspensful;
paranoid;
amnesiac;
samuri;
cliches;
revolutions;
erotica;
motel;hotels;
friend;
foods;
marihuana;
midieval;
1940s;1950s;1970s;1920s;
arts;
realist;
claustrophobia;claustraphobic;
werewolfs;
robots;
1950;1960s;1970s;1980s;1930s;1940s;
android;
students;
scare;
quoteable;
archeology;
1920;1930s;1960s;1940s;1900s;
flashbacks;
wizard;
gunfights;
reporters;
journalism;
nightmares;
shootouts;
sibling;
casinos;
1930s;1950s;1960s;1940;1970s;
paranoia;
blaxsploitation;
neighbour;
whodunnit;
spaceships;
geek;
funerals;
visuals;
terrorists;terrorism;
monster;
idealist;
propoganda;
brother;
comedians;
apocolyptic;
televison;
mo

impersonation;
incan;
impressionism;
inmate;
humanoid;
insider;
insiders;
insomnia;
harassment;
heroic;
jock;
j;
brazil;
camera;
camaraderie;
cameraman;
camper;
camaraderie;
bureaucrat;
cafe;
cadet;
biplane;
billiard;
blackout;
afi;
blaxploitation;
clans;
cliff;
cinematography;
commercial;
concept;
complaints;
complications;
collaborators;
collector;
colleague;
colonisation;
carrots;
cartel;
chimp;
chauvinist;
chauffeur;
chef;
alligator;
amateur;
allright;
animatronic;
amnesia;
ancestor;
80s;
1800s;1900s;
1820s;1800s;1890s;
1800s;1880s;1870s;
1867;
1863;
1800s;1890s;1870s;
1900s;1920;
1926;1932;
1936;1922;1920;
1936;1942;
1936;1939;1932;
1926;1935;1932;1939;
1950;1940s;1960;1920;1970;
1943;1940;1932;
1942;1940;
1985;1957;1959;
1957;1959;
1957;1958;1989;
1960s;1950;1970;1980;1963;
1963;1969;1960;
1968;1963;1960;
1969;1963;1966;
1968;1964;1963;1989;
60s;30s;90s;
1977;1979;1970;
1977;1970;
1985;1980;1989;1986;1987;
1986;1989;1984;1987;1981;
1987;1985;1989;1984;
1987;1985;1986;
1998;1997;1

In [16]:
tag_no_cap_punc_stem_word2vec = tag_no_cap_punc_stem.copy()
tag_no_cap_punc_stem_word2vec['word2vec'] = word2vec
tag_no_cap_punc_stem_word2vec['word2vec_2edits'] = word2vec_2edits

In [39]:
temp_df = tag_no_cap_punc_stem_word2vec[tag_no_cap_punc_stem_word2vec['duplicates'].notnull()].copy()


In [59]:
new_duplicates = []
for index, row in temp_df.iterrows():
    tag = row[0]
    new_dup_str = row[5]
    if new_dup_str != None:
        duplicates = new_dup_str.split(";")
        if row[7] != None:
            word2vec_2edits = row[7].split(";")
            for k in word2vec_2edits:
                if k not in duplicates:
                    new_dup_str = new_dup_str + k + ';'
        new_duplicates.append(new_dup_str);
    else:
        new_duplicates.append(None)

In [62]:
tag_no_cap_punc_stem_word2vec['duplicates_with_word2vec'] = tag_no_cap_punc_stem_word2vec['duplicates']
tag_no_cap_punc_stem_word2vec.loc[tag_no_cap_punc_stem_word2vec['duplicates'].notnull(), 'duplicates_with_word2vec'] = new_duplicates


### Tags table with all duplicates (capitalization, punctuation, and word2vec+2edits)

In [97]:
tag_no_cap_punc_stem_word2vec.head()

,tag,movieId,score,tag_stem,tags_same_stem,duplicates,word2vec,word2vec_2edits,duplicates_with_word2vec
0,sci-fi,30393804,7842.0,sci-fi,NaN,Sci-Fi;Sci-fi;#scifi;+scifi;,NaN,NaN,Sci-Fi;Sci-fi;#scifi;+scifi;
1,atmospheric,22351413,5399.0,atmospher,Atmosphere;,ATMOSPHERIC;Atmospheric;,moody;suspenseful;creepy;gloomy;,NaN,ATMOSPHERIC;Atmospheric;
2,surreal,30372701,4581.0,surreal,surrealism;,Surreal;,surrealistic;nightmarish;dreamlike;bizarre;sur...,NaN,Surreal;
3,twist ending,14940576,4102.0,twist end,twist endings;twist end;,Twist Ending;Twist ending;,NaN,NaN,Twist Ending;Twist ending;
4,visually appealing,20032372,3866.0,visually app,NaN,Visually Appealing;Visually appealing;,NaN,NaN,Visually Appealing;Visually appealing;


In [98]:
tag_no_cap_punc_stem_word2vec.to_csv("tag_no_cap_punc_stem_word2vec.csv",index=False)

In [107]:
tag_no_cap_punc_stem_word2vec = pd.read_csv("tag_no_cap_punc_stem_word2vec.csv")
data_to_insert = tag_no_cap_punc_stem_word2vec[["tag","score","duplicates","word2vec_2edits","duplicates_with_word2vec"]].copy()
data_to_insert.head()

,tag,score,duplicates,word2vec_2edits,duplicates_with_word2vec
0,sci-fi,7842.0,Sci-Fi;Sci-fi;#scifi;+scifi;,NaN,Sci-Fi;Sci-fi;#scifi;+scifi;
1,atmospheric,5399.0,ATMOSPHERIC;Atmospheric;,NaN,ATMOSPHERIC;Atmospheric;
2,surreal,4581.0,Surreal;,NaN,Surreal;
3,twist ending,4102.0,Twist Ending;Twist ending;,NaN,Twist Ending;Twist ending;
4,visually appealing,3866.0,Visually Appealing;Visually appealing;,NaN,Visually Appealing;Visually appealing;


In [108]:
data_to_insert = data_to_insert.where(data_to_insert.notnull(), None)

In [116]:
tuples = [tuple(x) for x in data_to_insert.values]


insert_to_sql(tuples, "tag_replacement")

/Users/evazhong/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1366, "Incorrect string value: '\\xC2\\x83\\xC3\\x82\\xC2\\xAD...' for column 'tag' at row 16077")
  result = self._query(query)
/Users/evazhong/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1366, "Incorrect string value: '\\xE5\\x8A\\xB1\\xE5\\xBF\\x97' for column 'tag' at row 19159")
  result = self._query(query)
/Users/evazhong/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1366, "Incorrect string value: '\\xE5\\x9D\\x9A\\xE5\\xBC\\xBA' for column 'tag' at row 19162")
  result = self._query(query)
/Users/evazhong/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1366, "Incorrect string value: '\\xE6\\x89\\xAD\\xE6\\x9B\\xB2' for column 'tag' at row 19207")
  result = self._query(query)
/Users/evazhong/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1366, "Incorrect string value: '\\xE5\\xA5\\x8B\\xE6\

/Users/evazhong/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1366, "Incorrect string value: '\\xCF\\x80\\xCE\\xBF\\xCF\\x85...' for column 'tag' at row 3496")
  result = self._query(query)
/Users/evazhong/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1366, "Incorrect string value: '\\xCF\\x88\\xCF\\x89\\xCF\\x86...' for column 'tag' at row 7365")
  result = self._query(query)
/Users/evazhong/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1366, "Incorrect string value: '\\xCF\\x84\\xCE\\xBF\\xCF\\x85...' for column 'tag' at row 7368")
  result = self._query(query)
/Users/evazhong/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1366, "Incorrect string value: '\\xCF\\x83\\xCE\\xBA\\xCE\\xB1...' for column 'tag' at row 7421")
  result = self._query(query)
/Users/evazhong/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (1366, "Incorrect string value: '\\xCF\\x80\\xCE\

In [114]:
### Connect and insert to SQL

def insert_to_sql(data_in_tuple, sql_table_name):
    import pymysql.cursors


    connection = pymysql.connect(host="127.0.0.1",
                                 user="web",
                                 password="atth1132",
                                 db="ML3_dev")
    try:
        with connection.cursor() as cursor:


            sql_drop = "DELETE from %s" % (sql_table_name)
            cursor.execute(sql_drop)
            sql = "insert ignore into tag_replacement(tag, score, duplicates,word2vec_2edits,duplicates_with_word2vec) VALUES(%s, %s, %s, %s, %s)" 
            #number_of_rows = cursor.executemany(sql, data_to_insert)
            #% (tag_list,score_list,duplicates_list,word2vec_2edits_list,duplicates_with_word2vec_list)
            #for x in tuples:   
            cursor.executemany(sql,data_in_tuple)
             #   cursor.execute(sql,x)

            connection.commit()
    finally:
        connection.close()


### A function that identifies if two words are <=1 edit away


In [11]:
def isEditDistanceTwo(s1, s2):
 
    # Find lengths of given strings
    m = len(s1)
    n = len(s2)
 
    # If difference between lengths is more than 1,
    # then strings can't be at one distance
    if abs(m - n) > 1:
        return False
 
    count = 0    # Count of isEditDistanceOne
 
    i = 0
    j = 0
    while i < m and j < n:
        # If current characters dont match
        if s1[i] != s2[j]:
            if count == 1:
                return False
 
            # If length of one string is
            # more, then only possible edit
            # is to remove a character
            if m > n:
                i+=1
            elif m < n:
                j+=1
            else:    # If lengths of both strings is same
                i+=1
                j+=1
 
            # Increment count of edits
            count+=1
 
        else:    # if current characters match
            i+=1
            j+=1
 
    # if last character is extra in any string
    if i < m or j < n:
        count+=1
 
    return count <= 2